In [10]:
import pandas as pd
import random
from copy import deepcopy
import numpy as np


In [11]:
import import_ipynb 
from accuracy import calc_accuracy

In [12]:
X_train = pd.read_csv('data/X_train.csv')
X_test  = pd.read_csv('data/X_test.csv')

y_train = pd.read_csv('data/y_train.csv')
y_test  = pd.read_csv('data/y_test.csv')

In [13]:
class Individual:
    def __init__(self,code,fitness = -float("inf")):
        self.code = code
        self.fitness = fitness

In [21]:
class GA:

    def __init__(self, X:pd.DataFrame, y:pd.DataFrame, population_size:int, num_generations:int, tournament_size:int, mutation_probability:float, alpha :float):
        #dodaj elitizam
        self.X = X
        self.y = y
        self.all_column_names = X.columns
        self.population_size = population_size
        self.num_generations = num_generations
        self.tournament_size = tournament_size
        self.mutation_probability = mutation_probability
        self.alpha = alpha
        
        self.population = [ Individual(code=(np.random.rand(X.shape[1]) < 0.25)) for _ in range(self.population_size) ]
        self.best_fitness = -float('inf')
        self.best = None
        self.history = []

    def calc_fitness(self,index): 
        if not any(self.population[index].code):
            self.population[index].fitness =  -float('inf')
            return
        
        new_column_names = [j for i,j in enumerate(self.all_column_names) if self.population[index].code[i]]
        acc = calc_accuracy(self.population[index].code,X_train,y_train,X_test,y_test)
        
        num_features =  sum(self.population[index].code)
        
        self.population[index].fitness = self.alpha * acc + (1 - self.alpha) * (1 - num_features / self.X.shape[1])   
        # a*tacnost + b * (1 - izabrani_atributi / ukupno_atributa )
        #ili a * tacnost + (1 - a) ...
        #mozda da oduzimam drugi deo
        

    def crossover(self,parent1,parent2):
        breakpoint = random.randrange(1, len(parent1.code))
        child1 = Individual(np.concatenate([parent1.code[:breakpoint],parent2.code[breakpoint:]]) )
        child2 =  Individual(np.concatenate([parent2.code[:breakpoint],parent1.code[breakpoint:]]) )

        return child1,child2

    def mutation(self,child):
        for i in range(len(child.code)):
            if random.random() < self.mutation_probability:
                child.code[i] = not child.code[i]
        return child
        
    def tournament_selection(self):
        contenders = random.sample(range(len(self.population)), self.tournament_size)
        return max(contenders, key=lambda i: self.population[i].fitness)
        

    def run(self):
               
        for generation in range(self.num_generations):
            
            for i in range(self.population_size):
                self.calc_fitness(i)
                if self.population[i].fitness > self.best_fitness:
                    self.best_fitness = self.population[i].fitness
                    self.best = self.population[i].code.copy()
            
            self.history.append((generation,self.best_fitness,self.best))
            new_population = []
            for i in range(0,self.population_size,2):
                parent1 = self.population[self.tournament_selection()]
                parent2 = self.population[self.tournament_selection()]
                child1, child2 = self.crossover(parent1, parent2)
                new_population.append(self.mutation(child1))
                if len(new_population) < self.population_size: #unapredi ovo
                    new_population.append(self.mutation(child2))
            self.population = deepcopy(new_population)

    

        return self.best, self.best_fitness #, history
    

In [22]:
ga = GA(X_train,y_train,population_size = 100, num_generations = 10, tournament_size=4, mutation_probability = 0.05,alpha = 0.5)
selected_features, fitness = ga.run()

In [23]:
fitness

1.760134661242811

In [24]:
len(X_train.columns[selected_features])

1710

In [25]:
len(X_train.columns)

7129

In [26]:
X_train.shape[1]

7129